# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [63]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr 
import sqlite3
import base64
from io import BytesIO

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
# You can choose whichever providers you like - or all Ollama

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
    

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCE


In [50]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [51]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (title TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [61]:
# Define the book prices tools
def get_book_price(book_title):
    print(f"DATABASE TOOL CALLED: Getting price for {book_title}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE title = ?', (book_title.lower(),))
        result = cursor.fetchone()
        return f"The price for book titled '{book_title}' is ₦{result[0]}" if result else "No price data available for this book_title"


def set_ticket_price(book_title, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (title, price) VALUES (?, ?) ON CONFLICT(title) DO UPDATE SET price = ?', (book_title.lower(), price, price))
        conn.commit()


book_prices = {
    "success buttons": "800", 
    "success strategies": "1000", 
    "following the path of the eagle": "1500", 
    "exploring the secrets of success": "700",
    "secrets of systems": "1000"
}

for book_title, price in book_prices.items():
    set_ticket_price(book_title, price)

In [62]:
get_book_price("success strategies")

DATABASE TOOL CALLED: Getting price for success strategies


"The price for book titled 'success strategies' is ₦1000.0"

In [ ]:

# There's a particular dictionary structure that's required to describe our function:
price_function = {
    "name": "get_book_price",
    "description": "Get the price of a book of the month by bishop david o. oyedepo",
    "parameters": {
        "type": "object",
        "properties": {
            "book_title": {
                "type": "string",
                "description": "The book that someone wants to buy",
            },
        },
        "required": ["book_title"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]

In [37]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_book_price":
            arguments = json.loads(tool_call.function.arguments)
            title = arguments.get('book_title')
            price_details = get_book_price(title)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses
    

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

In [38]:
system_prompt = """
                You are a helpful librarian at the dominion publish books store. You make helpful suggestion about the books available. 
                You especially recommend books of the month, that may have been announced during church services. Give a courteous answers. 
                Always be accurate. If you don't know the answer, say so.
                """


def chat_with_tool(message, history, model):

    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    if model == "GPT":
        response = openai.chat.completions.create(model='gpt-4.1-mini', messages=messages, tools=tools)
    else:
        response = anthropic.chat.completions.create(model='claude-sonnet-4-5-20250929', messages=messages, tools=tools)


    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        
        if model == "GPT":
            response = openai.chat.completions.create(model='gpt-4.1-mini', messages=messages, tools=tools)
        else:
            response = anthropic.chat.completions.create(model='claude-sonnet-4-5-20250929', messages=messages, tools=tools)
    
    
    return response.choices[0].message.content

In [ ]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")

view = gr.ChatInterface(
    fn=chat_with_tool, 
    title="Technical Assistant",
    additional_inputs=[model_selector], 
    type="messages",
    flagging_mode="never",
    js=force_dark_mode)

view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting price for Success Buttons
DATABASE TOOL CALLED: Getting price for Success Strategies
